In [1]:
# import libraries
import pandas as pd
df = pd.read_excel("digitale wereld submap.xlsx")
import re

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator

In [2]:
print(df.columns)

Index(['@digitaal_emoji', '@digitaal_watgaatergoed_categorie',
       '@digitaal_watgaatergoedd', '@digitaal_watkanerbeter_categorie',
       '@digitaal_watkanerbeter_extracategorie', '@digitaal_watkanerbeterr'],
      dtype='object')


In [3]:
df = df.drop(['@digitaal_watkanerbeter_extracategorie'], axis =1)
df = df.dropna()

df.head()

,@digitaal_emoji,@digitaal_watgaatergoed_categorie,@digitaal_watgaatergoedd,@digitaal_watkanerbeter_categorie,@digitaal_watkanerbeterr
0,😐,['Verbinding en contact'],Veel mogelijkheden ook voor contact met mensen...,"['Mentale gezondheid ', 'Verslaving']",Ik vind dat het afgeschaft moet worden.
1,😐,"['Sociale media', 'Verbinding en contact', 'Sn...","in NL overal prima goed bereikbaar, we kunnen ...","['Sociale media', 'Veiligheid', 'Verslaving', ...",schadelijke kant en negatieve effecten van soc...
2,😡,"['Toegankelijkheid', 'Anders, namelijk...']",Dat er de laatste jaren meer restricties zijn ...,"['Verslaving', 'Leeftijd', 'Invloed', 'Mentale...","Heel veel, het internet neemt mensen nu over. ..."
3,😐,"['Toegang tot kennis en informatie', 'Verbindi...",Kennis die je snel hebt waardoor je veel kan l...,"['Invloed', 'Nepnieuws/ Desinformatie', 'Priva...",Het is teveel verslavend en er staan teveel nu...
6,😡,"['Bewustzijn', 'Sociale media', 'Toegang tot k...","De mogelijkheid om te kunnen communiceren, spe...","['Bewustzijn', 'Privacy', 'Toegankelijkheid', ...",Er is veel media die bepaalde doelgroepen of m...


In [69]:
def analyze_dutch_sentiment(dutch_sentence):
    if not re.search(r'[a-zA-Z]', dutch_sentence):
        return {
            'original': dutch_sentence,
            'translated': None,
            'sentiment': 'Invalid - No words',
            'scores': None
        }
    # Step 1: Translate Dutch to English
    english_sentence = GoogleTranslator(source='nl', target='en').translate(dutch_sentence)

    # Step 2: Analyze sentiment with VADER
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(english_sentence)

    # Step 3: Classify based on compound score
    compound = sentiment_scores['compound']
    if compound >= 0.1:
        sentiment = 'Positive'
    elif compound <= 0.1:
        sentiment = 'Negative'

    return {
        'original': dutch_sentence,
        'translated': english_sentence,
        'sentiment': sentiment,
        'scores': sentiment_scores
    }

# Example
result = analyze_dutch_sentiment("Ik haat dit product, het is geweldig!")
print(result)


{'original': 'Ik haat dit product, het is geweldig!', 'translated': "I hate this product, it's great!", 'sentiment': 'Positive', 'scores': {'neg': 0.306, 'neu': 0.331, 'pos': 0.363, 'compound': 0.1759}}


In [18]:
for e in df['@digitaal_watgaatergoedd'].to_list():
    print(analyze_dutch_sentiment(e)['sentiment'], e)

Neutral Veel mogelijkheden ook voor contact met mensen ver weg
Positive in NL overal prima goed bereikbaar, we kunnen werken overal vandaan, ik kan eenvoudig met iedereen in contact blijven
Neutral Dat er de laatste jaren meer restricties zijn voor jonge mensen. _x000D_

Neutral Kennis die je snel hebt waardoor je veel kan leren. Verbinding ondanks lange afstanden
Positive De mogelijkheid om te kunnen communiceren, spelen, dingen online te delen etc. met anderen over de hele wereld dankzij de technologie is positief.
Negative Het verbindt mensen met elkaar ook degene die normaal niet naar buiten kunnen vanwege hun medische/ phygische situatie en geeft hen een platform om toch zich te verbinden of een plek te vinden waar ze hun geld mee kunnen verdienen (kijk naar ironmouse, de jongen van world of war craft etc.)
Positive Ik vind het fijn dat je zo makkelijk andere mensen kan bereiken
Neutral Door ontwikkelingen in de digitale wereld, kunnen mensen steeds eenvoudiger met elkaar in conta

Positive Ik vind het fijn dat de digitale wereld er voor zorgt dat meer dingen geautomatiseerd worden en daarmee efficiënter en gemakkelijker toegankelijk 
Neutral hi
Neutral Mogelijkheden om contact te hebben met mensen overal en altijd
Neutral de mogelijkheid dat t r is_x000D_

Positive Het is super handig dat je snel informatie kan vinden op internet oid. En snel kan communiceren met je netwerk 


In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# Load tokenizer and model
model_name = "pdelobelle/robbert-v2-dutch-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    "pdelobelle/robbert-v2-dutch-base", num_labels=2
)

# Label mapping based on the fine-tuned RobBERT classifier
labels = ["negatief", "positief"]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at pdelobelle/robbert-v2-dutch-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
def analyze_sentiment_robbert(dutch_sentence):
    # Tokenize input
    inputs = tokenizer(dutch_sentence, return_tensors="pt", truncation=True, padding=True)
    
    # Get model output
    with torch.no_grad():
        outputs = model(**inputs)
        
    # Compute probabilities
    probs = F.softmax(outputs.logits, dim=1)[0]
    
    # Get sentiment
    sentiment_idx = torch.argmax(probs).item()
    sentiment = labels[sentiment_idx]
    
    return {
        "sentence": dutch_sentence,
        "sentiment": sentiment,
        "probabilities": {labels[i]: float(probs[i]) for i in range(len(labels))}
    }

# Example
print(analyze_sentiment_robbert("Heel erg handig"))


Heel erg handig
{'sentence': 'Heel erg handig', 'sentiment': 'positief', 'probabilities': {'negatief': 0.4169086813926697, 'positief': 0.5830913186073303}}


In [46]:
for e in df['@digitaal_watgaatergoedd'].to_list():
    if analyze_sentiment_robbert(e)['sentiment'] == 'negatief':
        print(analyze_sentiment_robbert(e)['sentiment'], e)


This one is negative
negatief Chatgpt
This one is negative
negatief Gepest over sociale media 


In [75]:
for e in df['@digitaal_watkanerbeterr'].to_list():
    if analyze_dutch_sentiment(e)['sentiment'] == 'Positive':
        print(analyze_dutch_sentiment(e)['sentiment'],e)

Positive Heel veel, het internet neemt mensen nu over. Mensen kunnen er niet meer zonder en worden er enorm door beïnvloed. Er wordt wel gezegd dat mensen zo fantastisch zijn en zoveel kunnen, maar die zelfde mensen zijn verslaafd aan hulpmiddelen van het internet en zijn zelf steeds minder creatief._x000D_
_x000D_
Wij hechten TE veel waarde aan het internet. Dit moet echt veranderen, want met de generatie worden kinderen minder intelligent. 
Positive Privacy, en de 'cancel cultuur', zomaar mensen hun leven kunnen verpesten omdat iemand iets fout gedaan zou hebben soms is het terecht maar niet altijd, het is namelijk moeilijk om te vinden wat wel echt waar is en wat niet. Iedereen kan zeggen en doen wat ze willen, dit gaat soms wel te ver. Ik ben voor vrijheid van meningsuiting maar niet ten koste van andere. 
Positive Veiligheid
Positive Er moet veel meer gelet worden wat onder de normale gebruiker loopt. Zo zijn er verschijdene apps die onder de ijsberg veel illegale dingen doen, het